<a href="https://colab.research.google.com/github/rnanclarez/scripts_mestrado/blob/main/NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementações parcialmente baseadas nas seguintes fontes
https://www.analyticsvidhya.com/blog/2021/05/bert-for-natural-language-inference-simplified-in-pytorch/'

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
### informação do tipo de processador usado(cpu/gpu...)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

### informação de uso de memória
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
%%script false
!pip install -U torch==1.10.0 torchtext==0.11.0
!cp -r /content/drive/MyDrive/mestrado/colab/kfold ./kfold
!cp -r /content/drive/MyDrive/mestrado/colab/nlisint ./nlisint
!pip install transformers


In [ ]:
from nlisint import utils as ut

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)

In [7]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM, get_constant_schedule_with_warmup

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)


In [ ]:
# Load pre-trained model (weights)
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [9]:
cls_token = ut.tokenizer.cls_token
sep_token = ut.tokenizer.sep_token
pad_token = ut.tokenizer.pad_token
unk_token = ut.tokenizer.unk_token
#print(cls_token, sep_token, pad_token, unk_token)

In [10]:
cls_token_idx = ut.tokenizer.cls_token_id
sep_token_idx = ut.tokenizer.sep_token_id
pad_token_idx = ut.tokenizer.pad_token_id
unk_token_idx = ut.tokenizer.unk_token_id
#print(cls_token_idx, sep_token_idx, pad_token_idx, unk_token_idx)

In [11]:
#For latest version use torchtext.legacy
from torchtext.legacy import data
#For sequence
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = ut.split_and_cut,
                  preprocessing = ut.tokenizer.convert_tokens_to_ids,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)
#For label
LABEL = data.LabelField()
#For Attention mask
ATTENTION = data.Field(batch_first = True,
                       use_vocab = False,
                       tokenize = ut.split_and_cut,
                       preprocessing = ut.convert_to_int,
                       pad_token = pad_token_idx)
#For token type ids
TTYPE = data.Field(batch_first = True,
                   use_vocab = False,
                   tokenize = ut.split_and_cut,
                   preprocessing = ut.convert_to_int,
                   pad_token = 1)

In [12]:
fields = [('label', LABEL), ('sequence', TEXT), ('attention_mask', ATTENTION), ('token_type', TTYPE)]

In [30]:
FOLD = 5
#print(f'snli_1.0_train_k{FOLD}.csv')

In [31]:
#train_data, valid_data, test_data = data.TabularDataset.splits(path = '/kfold/coliee',
#train_data, valid_data = data.TabularDataset.splits(path = '/content/kfold/coliee',
train_data, valid_data = data.TabularDataset.splits(path = '/content/kfold/snli_binario',

                                        train = f'snli_bi_1.0_train_k{FOLD}.csv',
                                        #train = f'coliee4_train_k{FOLD}.csv',
                                        validation = f'snli_bi_1.0_valid_k{FOLD}.csv',
                                        #validation = f'coliee4_valid_k{FOLD}.csv',
                                        #test = 'snli_1.0_test_full.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True)

In [32]:
LABEL.build_vocab(train_data)

In [33]:
#Create iterator
BATCH_SIZE = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
train_iterator, valid_iterator = data.BucketIterator.splits(
#   (train_data, valid_data, test_data), 
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.sequence),
    sort_within_batch = False, 
    device = device)

In [17]:
import torch.nn as nn
class BERTNLIModel(nn.Module):
    def __init__(self,
                 bert_model,
                 hidden_dim,
                 output_dim,):
        super().__init__()
        self.bert = bert_model

        embedding_dim = bert_model.config.to_dict()['hidden_size']
        self.out = nn.Linear(embedding_dim, output_dim)
    def forward(self, sequence, attn_mask, token_type):
        embedded = self.bert(input_ids = sequence, attention_mask = attn_mask, token_type_ids= token_type)[1]
        output = self.out(embedded)
        return output

In [18]:
#defining model
HIDDEN_DIM = 512
OUTPUT_DIM = len(LABEL.vocab)
model = BERTNLIModel(bert_model, HIDDEN_DIM, OUTPUT_DIM, ).to(device)

In [19]:
import torch.optim as optim
#optimizer = optim.AdamW(model.parameters(),lr=2e-5,eps=1e-6,correct_bias=False)
optimizer = optim.AdamW(model.parameters(),lr=2e-5,eps=1e-6)
def get_scheduler(optimizer, warmup_steps):
    scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)
    return scheduler
criterion = nn.CrossEntropyLoss().to(device)
#mp = True
mp = False

In [20]:
#!git clone https://github.com/NVIDIA/apex
#!!cd apex
#!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
#!cd ..

In [21]:
#if mp:
#    try:
#        from apex import amp
#    except ImportError:
#        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
#    model, optimizer = amp.initialize(model, optimizer, opt_level='O1')

In [22]:
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True)

    correct = (max_preds.squeeze(1)==y).float()

    return correct.sum() / len(y)

In [23]:
max_grad_norm = 1
def train(model, iterator, optimizer, criterion, scheduler):
  epoch_loss = 0
  epoch_acc = 0
  model.train()
  for batch in iterator:
    optimizer.zero_grad() # clear gradients first
    torch.cuda.empty_cache() # releases all unoccupied cached memory
    sequence = batch.sequence
    attn_mask = batch.attention_mask
    token_type = batch.token_type
    label = batch.label
    predictions = model(sequence, attn_mask, token_type)
    loss = criterion(predictions, label)
    acc = categorical_accuracy(predictions, label)
    if mp:
      with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
        torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
    else:
      loss.backward()
      optimizer.step()
      scheduler.step()
      epoch_loss += loss.item()
      epoch_acc += acc.item()
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
def evaluate(model, iterator, criterion):
  epoch_loss = 0
  epoch_acc = 0
  model.eval()
  with torch.no_grad():
    for batch in iterator:
      sequence = batch.sequence
      attn_mask = batch.attention_mask
      token_type = batch.token_type
      labels = batch.label
      predictions = model(sequence, attn_mask, token_type)
      loss = criterion(predictions, labels)
      acc = categorical_accuracy(predictions, labels)
      epoch_loss += loss.item()
      epoch_acc += acc.item()
  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [25]:
import time
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [26]:
train_data_len = 335680

In [27]:
import math
N_EPOCHS = 5
warmup_percent = 0.2
total_steps = math.ceil(N_EPOCHS*train_data_len*1./BATCH_SIZE)
warmup_steps = int(total_steps*warmup_percent)
scheduler = get_scheduler(optimizer, warmup_steps)
best_valid_loss = float('inf')

In [ ]:
print(f'fold {FOLD}')
for epoch in range(N_EPOCHS):
  start_time = time.time()
  train_loss, train_acc = train(model, train_iterator, optimizer, criterion, scheduler)
  valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'bert-nli.pt')
  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f't Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')

fold 5


In [29]:
!cp ./bert-nli.pt /content/drive/MyDrive/mestrado/colab/modelosTreinados/bert_snli_BI_k04.pt

In [ ]:
!cp /content/drive/MyDrive/mestrado/colab/modelosTreinados/bert_snli_BI_k4.pt .

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/mestrado/colab/modelosTreinados/bert_snli_BI_k4.pt'))
#model.load_state_dict(torch.load('bert-nli.pt',map_location=torch.device('cpu') ))
#model.load_state_dict(torch.load('bert_snli_BI_k4.pt'))
#test_loss, test_acc = evaluate(model, test_iterator, criterion)
test_loss, test_acc = evaluate(model, valid_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')


Test Loss: 1.459 |  Test Acc: 52.82%


In [ ]:
def predict_inference(premise, hypothesis, model, device):
    model.eval()
    premise = '[CLS] ' + premise + ' [SEP]'
    hypothesis = hypothesis + ' [SEP]'
    prem_t = ut.tokenize_bert(premise)
    hypo_t = ut.tokenize_bert(hypothesis)
    prem_type = ut.get_sent1_token_type(prem_t)
    hypo_type = ut.get_sent2_token_type(hypo_t)
    indexes = prem_t + hypo_t
    indexes = ut.tokenizer.convert_tokens_to_ids(indexes)
    indexes_type = prem_type + hypo_type
    attn_mask = ut.get_sent2_token_type(indexes)
    indexes = torch.LongTensor(indexes).unsqueeze(0).to(device)
    indexes_type = torch.LongTensor(indexes_type).unsqueeze(0).to(device)
    attn_mask = torch.LongTensor(attn_mask).unsqueeze(0).to(device)
    prediction = model(indexes, attn_mask, indexes_type)
    prediction = prediction.argmax(dim=-1).item()
    return LABEL.vocab.itos[prediction]

In [ ]:
premise = 'john and his brother is wallking in the park'
hypothesis = 'john has a sister'
predict_inference(premise, hypothesis, model, device)